In [318]:
from memory_profiler import memory_usage
import sys
import os
import warnings
from datetime import datetime
import datetime as dt
from fastapi import FastAPI, Query, HTTPException
from fastapi.staticfiles import StaticFiles
from starlette.responses import FileResponse
import boto3
import polars as pl
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import io
warnings.filterwarnings('ignore')

import psutil

def get_interventions_political_group_per_day2(
    name: str,
    start_date_: str = Query(default = start_date, description="Start date"),
    end_date_: str = Query(default = end_date, description="End date"),
    kind_: str = Query(default = "both" , description="Type of data", enum = kind),
    channel_: str = Query(default = "all", description="Channel"),
    program_: str = Query(default = "all", description="Program"),
    topic_: str = Query(default = "all", description="Topic"),
    page: int = Query(default=1, description="Page number"),
    page_size: int = Query(default=10, description="Page size")
):
    """
    Return how much a political group has intervened in tv per day
    """
    filtered_data = political_groups

    if name not in political_groups_list:
        raise HTTPException(status_code=400, detail="Political Group not found")
    filtered_data = filtered_data.filter(pl.col('lastname') == name)

    if channel_ != "all":
        if channel_ not in channels:
            raise HTTPException(status_code=400, detail="Invalid channel")
        filtered_data = filtered_data.filter(pl.col('channel') == channel_)

    if program_ != "all":
        if program_ not in programs:
            raise HTTPException(status_code=400, detail="Invalid program")
        filtered_data = filtered_data.filter(pl.col('program') == program_)

    if topic_ != "all":
        if topic_ not in topics:
            raise HTTPException(status_code=400, detail="Invalid topic")
        filtered_data = filtered_data.filter(pl.col('topic') == topic_)

    interventions_polgroup = filter_data(filtered_data, start_date_, end_date_, kind_)
    start = (page - 1) * page_size
    end = start + page_size

    begin_date = dt.date(int(start_date_.split('/')[0]),
                         int(start_date_.split('/')[1]),
                         int(start_date_.split('/')[2]))
    b = begin_date
    final_date = dt.date(int(end_date_.split('/')[0]),
                         int(end_date_.split('/')[1]),
                         int(end_date_.split('/')[2]))
    day = dt.timedelta(1)
    year = begin_date.year
    interventions = []
    i = {}
    max_value = 0
    total_lines = interventions_polgroup.collect().height

    offset = 0  # Iniziamo dalla prima riga
    # # print(interventions_polgroup.collect())
    # print(interventions_polgroup.slice(16, 1).collect())

    for j in range(total_lines):
        row = interventions_polgroup.slice(j, 1).collect()

        if row.height != 0:  # Se non ci sono più righe, fermiamo il ciclo
            s_day = row["day"].item().strftime('%Y/%m/%d')
            if s_day in i:
                i[s_day] += 1
            else:
                i[s_day] = 1
        print(j)
    print(i)

# 0 - 1 - 18 - 37 - 47 - 65 - 75 - 165
#  1   17   19   10   18   10   90

    return ""
get_interventions_political_group_per_day2("Partito Democratico", start_date, end_date, "both", "all", "all", "all", 1, 10)

In [319]:
data = pl.scan_parquet(".." + os.sep + ".." + os.sep + "docs"  + os.sep +  "data" + os.sep + "agcomdata.parquet")

In [320]:
data = data.with_columns([
    (pl.col("name") + " " + pl.col("lastname")).alias("fullname"),
    pl.col("kind").replace({"Parola": "speech", "Notizia": "news"}).alias("kind")
])

politicians = data.filter(pl.col("name") != "Soggetto collettivo")
politicians_list = politicians.select("fullname").unique().collect().to_series().to_list()

political_groups = data.filter(pl.col("name") == "Soggetto collettivo")
political_groups_list = political_groups.select('lastname').unique().collect().to_series().to_list()

start_date = data.select(pl.min("day")).collect().to_series()[0].strftime('%Y/%m/%d')
end_date = data.select(pl.max("day")).collect().to_series()[0].strftime('%Y/%m/%d')

kind = ["speech", "news", "both"]

channels = data.select("channel").unique().collect().to_series().to_list()
programs = data.select('program').unique().collect().to_series().to_list()
affiliations = data.select('affiliation').unique().collect().to_series().to_list()
topics = data.select('topic').unique().collect().to_series().to_list()

In [321]:
def validate_date(date_str):
    """
    Function to validate dates (from string to datetime)
    """
    try:
        return datetime.strptime(date_str, '%Y/%m/%d')
    except ValueError as exc:
        raise HTTPException(status_code=400, detail="Invalid date format. Use YYYY/MM/DD") from exc


def filter_data(data_, start_date_, end_date_, kind_):
    """
    Function to filter data by selected date interval and kind
    """
    start_date_ = validate_date(start_date_)
    end_date_ = validate_date(end_date_)

    if start_date_ > end_date_:
        raise HTTPException(status_code=400, detail="Start date must be before end date")

    filtered_data = data_.filter(pl.col("day") >= start_date_).filter(pl.col("day") <= end_date_)


    if kind_ != "both":
        filtered_data = filtered_data.filter(pl.col("kind") == kind_)

    return filtered_data

In [322]:
def get_interventions_political_group_per_day(
    name: str,
    start_date_: str = Query(default = start_date, description="Start date"),
    end_date_: str = Query(default = end_date, description="End date"),
    kind_: str = Query(default = "both" , description="Type of data", enum = kind),
    channel_: str = Query(default = "all", description="Channel"),
    program_: str = Query(default = "all", description="Program"),
    topic_: str = Query(default = "all", description="Topic"),
    page: int = Query(default=1, description="Page number"),
    page_size: int = Query(default=10, description="Page size")
):
    """
    Return how much a political group has intervened in tv per day
    """
    filtered_data = political_groups

    if name not in political_groups_list:
        raise HTTPException(status_code=400, detail="Political Group not found")
    filtered_data = filtered_data.filter(pl.col('lastname') == name)

    if channel_ != "all":
        if channel_ not in channels:
            raise HTTPException(status_code=400, detail="Invalid channel")
        filtered_data = filtered_data.filter(pl.col('channel') == channel_)

    if program_ != "all":
        if program_ not in programs:
            raise HTTPException(status_code=400, detail="Invalid program")
        filtered_data = filtered_data.filter(pl.col('program') == program_)

    if topic_ != "all":
        if topic_ not in topics:
            raise HTTPException(status_code=400, detail="Invalid topic")
        filtered_data = filtered_data.filter(pl.col('topic') == topic_)

    interventions_polgroup = filter_data(filtered_data, start_date_, end_date_, kind_)
    total_lines = interventions_polgroup.collect().height
    batch_size = 5000

    begin_date = dt.date(int(start_date_.split('/')[0]),
                         int(start_date_.split('/')[1]),
                         int(start_date_.split('/')[2]))
    final_date = dt.date(int(end_date_.split('/')[0]),
                         int(end_date_.split('/')[1]),
                         int(end_date_.split('/')[2]))
    day = dt.timedelta(1)

    first_year = begin_date.year
    last_year = final_date.year

    calendar_list = []

    batch_index = 0
    batch = interventions_polgroup.slice(batch_index, batch_size).collect()

    while batch.height != 0:
        calendar_list.extend(batch.select("day").to_series().to_list())
        batch_index += batch_size
        batch = interventions_polgroup.slice(batch_index, batch_size).collect()

    return 0
get_interventions_political_group_per_day("Partito Democratico", start_date, end_date, "both", "all", "all", "all", 1, 10)
mem_usage = memory_usage(
    (get_interventions_political_group_per_day, 
     ("Partito Democratico", start_date, end_date, "both", "all", "all", "all", 1, 10))
)

print(f"Memoria utilizzata: {max(mem_usage) - min(mem_usage)} MiB")
print(get_interventions_political_group_per_day("Partito Democratico", start_date, end_date, "both", "all", "all", "all", 1, 10))

Memoria utilizzata: 183.19921875 MiB
0
